In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import os
from torch.utils.data import Dataset, DataLoader , random_split
from torchvision import transforms , datasets
from PIL import Image
import pandas as pd
import re
import random
import shutil
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

In [2]:
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not availabe")


CUDA is available!


In [3]:
meta_files = os.listdir("/home/jaskaran/data_selection/meta_file")
dfs = []

for file in meta_files:
    df = pd.read_csv(os.path.join("/home/jaskaran/data_selection/meta_file",file),sep="\t")
    dfs.append(df)

# Concatenate all DataFrames in the dfs list into one DataFrame
combined_df_sel = pd.concat(dfs, ignore_index=True)

# Display the combined DataFrame
print(combined_df_sel)

# If you want to save the combined DataFrame to a new CSV file
combined_df_sel.to_csv("combined_meta_file_sel.csv", index=False)

      selection_coefficient  \
0                    0.0012   
1                    0.0149   
2                    0.0393   
3                    0.0500   
4                    0.0744   
...                     ...   
4995                 0.8760   
4996                 0.9307   
4997                 0.9369   
4998                 0.9629   
4999                 0.9932   

                                             fasta_file  Tajimas_D  Tajima_pi  \
0     output/output_selection/fasta/selection_0.0012...  -0.039485   0.002144   
1     output/output_selection/fasta/selection_0.0149...  -0.945775   0.000954   
2     output/output_selection/fasta/selection_0.0393...  -2.035884   0.000433   
3     output/output_selection/fasta/selection_0.05.f...  -2.107024   0.000416   
4     output/output_selection/fasta/selection_0.0744...  -2.518223   0.000219   
...                                                 ...        ...        ...   
4995  output/output_selection/fasta/selection_0.876....  -2.

In [4]:
for file in os.listdir("/home/jaskaran/data_selection/txt_file"):
    file_path = os.path.join("/home/jaskaran/data_selection/txt_file",file)
    print(file_path)
    with open(file_path, "r") as file:
        content = file.read()
    lines = content.split("\n")

    start_index = lines.index(" The allele frequency numbers are :-  ") + 1


    end_index = next(i for i, line in enumerate(lines) if line.startswith("1. The Tajima'D for the given sequence is"))

    sfs = [num for num in lines[start_index:end_index]]
    sfs = " ".join(sfs)
    sfs = sfs[1:len(sfs)-1]
    sfs = sfs.split()
    sfs = list(map(float, sfs))
    category = [0]*9
    category[0] = sum(sfs[0:1])
    category[1] = sum(sfs[1:2])
    category[2] = sum(sfs[2:4])
    category[3] = sum(sfs[4:7])
    category[4] = sum(sfs[7:10])
    category[5] = sum(sfs[10:20])
    category[6] = sum(sfs[20:50])
    category[7] = sum(sfs[50:70])
    category[8] = sum(sfs[70:])
    find = re.match(r".+?[_].+?[_].+?[_](.+?)[.][t][x][t]",file_path)
    coef = float(find[1])
    j=1
    for i in category:
        combined_df_sel.loc[combined_df_sel["selection_coefficient"]==coef,f"category_sfs_{j}"] = i
        j+=1
    

/home/jaskaran/data_selection/txt_file/selection_0.4211.txt
/home/jaskaran/data_selection/txt_file/selection_0.0263.txt
/home/jaskaran/data_selection/txt_file/selection_0.5576.txt
/home/jaskaran/data_selection/txt_file/selection_0.1972.txt
/home/jaskaran/data_selection/txt_file/selection_0.57.txt
/home/jaskaran/data_selection/txt_file/selection_0.3694.txt
/home/jaskaran/data_selection/txt_file/selection_0.0841.txt
/home/jaskaran/data_selection/txt_file/selection_0.0122.txt
/home/jaskaran/data_selection/txt_file/selection_0.0666.txt
/home/jaskaran/data_selection/txt_file/selection_0.5044.txt
/home/jaskaran/data_selection/txt_file/selection_0.6515.txt
/home/jaskaran/data_selection/txt_file/selection_0.1499.txt
/home/jaskaran/data_selection/txt_file/selection_0.9202.txt
/home/jaskaran/data_selection/txt_file/selection_0.7977.txt
/home/jaskaran/data_selection/txt_file/selection_0.4615.txt
/home/jaskaran/data_selection/txt_file/selection_0.7548.txt
/home/jaskaran/data_selection/txt_file/sel

In [5]:
len(combined_df_sel.dropna())

5000

In [6]:
for i in combined_df_sel["selection_coefficient"]:
    print(i)

0.0012
0.0149
0.0393
0.05
0.0744
0.0774
0.0881
0.1018
0.1399
0.1994
0.2024
0.2131
0.2375
0.2756
0.2844
0.2875
0.2893
0.3
0.4006
0.4094
0.425
0.4631
0.4768
0.4875
0.4982
0.5371
0.5375
0.5437
0.5607
0.5637
0.5881
0.6506
0.6718
0.7131
0.7649
0.7875
0.7906
0.7968
0.8107
0.8381
0.85
0.8562
0.8593
0.8732
0.9094
0.9156
0.9187
0.9218
0.9843
0.9982
0.0858
0.0877
0.0965
0.1971
0.2352
0.2459
0.2596
0.2659
0.2752
0.2783
0.284
0.2977
0.3221
0.3346
0.3377
0.3909
0.4002
0.4227
0.4627
0.4852
0.5097
0.5159
0.5722
0.5753
0.5815
0.5877
0.5908
0.6378
0.6727
0.6941
0.7003
0.7034
0.7313
0.7566
0.7628
0.7659
0.769
0.7721
0.7752
0.7977
0.8377
0.8501
0.8816
0.8878
0.8909
0.9033
0.9334
0.9472
0.9534
0.9565
0.006
0.0167
0.0411
0.0548
0.0792
0.0929
0.1173
0.1417
0.1447
0.1798
0.2042
0.2179
0.2316
0.2667
0.2697
0.3048
0.3078
0.3185
0.3322
0.378
0.3917
0.4054
0.4191
0.4298
0.4328
0.4542
0.4816
0.4923
0.4953
0.5266
0.5441
0.5548
0.586
0.5891
0.5922
0.6066
0.6516
0.7072
0.711
0.7172
0.7697
0.7766
0.8048
0.8391
0.8422

In [7]:
root_folder = "/home/jaskaran/data_selection/temp_slim"
list_files = os.listdir(root_folder)
data = []
for i in list_files:
    find = re.match(r".+?[_].+?[_].+?[_](.+?)[.][s][l][i][m]",os.path.join(root_folder,i) )
    with open(os.path.join(root_folder,i)) as txt_file:
        text = txt_file.readlines()
        for str in text:
            try:
                index = str.index("target.addNewDrawnMutation(m2, ")
                if index == 4:
                   pos_start = index + len("target.addNewDrawnMutation(m2, ")
                   pos_end = str.index(")", pos_start)
                   position = int(str[pos_start:pos_end])
                   data.append({"selection_coefficient": float(find[1]), "Position": position})
            except ValueError:
                next

df = pd.DataFrame(data)
print(df)

      selection_coefficient  Position
0                    0.2925     88217
1                    0.0212     58782
2                    0.3884     20955
3                    0.2770     40550
4                    0.1188     85460
...                     ...       ...
5015                 0.0908     24692
5016                 0.6074     95873
5017                 0.0809     99139
5018                 0.8379     78586
5019                 0.1610     31932

[5020 rows x 2 columns]


In [8]:
combined_df = pd.merge(combined_df_sel, df, on='selection_coefficient')
combined_df

,selection_coefficient,fasta_file,Tajimas_D,Tajima_pi,num_of_mutation,snps_file,matrix_image_file,ancestral_seq,snps_image,output_file,...,category_sfs_1,category_sfs_2,category_sfs_3,category_sfs_4,category_sfs_5,category_sfs_6,category_sfs_7,category_sfs_8,category_sfs_9,Position
0,0.0012,output/output_selection/fasta/selection_0.0012...,-0.039485,0.002144,1123.0,output/output_selection/snps/snps_0.0012.fasta,output/output_selection/matrix/matrix_0.0012.txt,output/output_selection/ancestral_sequence/anc...,output/output_selection/images/image_0.0012.png,output/output_selection/h_result/h_0.0012.txt,...,0.077471,0.073019,0.063224,0.102404,0.036509,0.144256,0.150490,0.057881,0.077471,34309
1,0.0149,output/output_selection/fasta/selection_0.0149...,-0.945775,0.000954,684.0,output/output_selection/snps/snps_0.0149.fasta,output/output_selection/matrix/matrix_0.0149.txt,output/output_selection/ancestral_sequence/anc...,output/output_selection/images/image_0.0149.png,output/output_selection/h_result/h_0.0149.txt,...,0.128655,0.080409,0.083333,0.029240,0.027778,0.052632,0.127193,0.033626,0.169591,25980
2,0.0393,output/output_selection/fasta/selection_0.0393...,-2.035884,0.000433,560.0,output/output_selection/snps/snps_0.0393.fasta,output/output_selection/matrix/matrix_0.0393.txt,output/output_selection/ancestral_sequence/anc...,output/output_selection/images/image_0.0393.png,output/output_selection/h_result/h_0.0393.txt,...,0.160714,0.083929,0.080357,0.083929,0.021429,0.017857,0.000000,0.000000,0.258929,90979
3,0.0500,output/output_selection/fasta/selection_0.05.f...,-2.107024,0.000416,567.0,output/output_selection/snps/snps_0.05.fasta,output/output_selection/matrix/matrix_0.05.txt,output/output_selection/ancestral_sequence/anc...,output/output_selection/images/image_0.05.png,output/output_selection/h_result/h_0.05.txt,...,0.139330,0.072310,0.065256,0.026455,0.010582,0.037037,0.014109,0.007055,0.241623,84103
4,0.0744,output/output_selection/fasta/selection_0.0744...,-2.518223,0.000219,441.0,output/output_selection/snps/snps_0.0744.fasta,output/output_selection/matrix/matrix_0.0744.txt,output/output_selection/ancestral_sequence/anc...,output/output_selection/images/image_0.0744.png,output/output_selection/h_result/h_0.0744.txt,...,0.149660,0.138322,0.124717,0.011338,0.018141,0.006803,0.000000,0.000000,0.181406,8098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.8760,output/output_selection/fasta/selection_0.876....,-2.710532,0.000102,271.0,output/output_selection/snps/snps_0.876.fasta,output/output_selection/matrix/matrix_0.876.txt,output/output_selection/ancestral_sequence/anc...,output/output_selection/images/image_0.876.png,output/output_selection/h_result/h_0.876.txt,...,0.206642,0.107011,0.099631,0.014760,0.000000,0.007380,0.000000,0.000000,0.000000,14359
4996,0.9307,output/output_selection/fasta/selection_0.9307...,-2.564993,0.000197,421.0,output/output_selection/snps/snps_0.9307.fasta,output/output_selection/matrix/matrix_0.9307.txt,output/output_selection/ancestral_sequence/anc...,output/output_selection/images/image_0.9307.png,output/output_selection/h_result/h_0.9307.txt,...,0.199525,0.090261,0.111639,0.035629,0.007126,0.000000,0.000000,0.000000,0.156770,54263
4997,0.9369,output/output_selection/fasta/selection_0.9369...,-2.727486,0.000094,256.0,output/output_selection/snps/snps_0.9369.fasta,output/output_selection/matrix/matrix_0.9369.txt,output/output_selection/ancestral_sequence/anc...,output/output_selection/images/image_0.9369.png,output/output_selection/h_result/h_0.9369.txt,...,0.171875,0.082031,0.050781,0.054688,0.003906,0.000000,0.000000,0.000000,0.046875,68846
4998,0.9629,output/output_selection/fasta/selection_0.9629...,-2.748873,0.000103,287.0,output/output_selection/snps/snps_0.9629.fasta,output/output_selection/matrix/matrix_0.9629.txt,output/output_selection/ancestral_sequence/anc...,output/output_selection/images/image_0.9629.png,output/output_selection/h_resul

In [9]:
class GeneticSelectionData:
    def __init__(self,root_folder,data_table,transform= None):
        self.root_folder = root_folder
        self.data_table = data_table
        self.file_list = self._build_file_list()
        self.transform = transform
    
    def _build_file_list(self):
        file_list = []
        for image_file in os.listdir(self.root_folder):
            image_path = os.path.join(self.root_folder,image_file)
            file_list.append(image_path)
        return file_list
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self,index):
        file_path = self.file_list[index]
        image = Image.open(file_path)
        image = image.convert("RGB")
        find = re.match(r".+?[_].+?[_](.+?)[.][p][n][g]", file_path)
        coef = float(find[1])
        if self.transform:
            image = self.transform(image)
        Position = self.data_table[self.data_table["selection_coefficient"] == coef]["Position"]
        try :
            Position = Position.values[0]
        except IndexError:
            Position = 8000
        if coef <= 0.33:
             coef_class = 0
        elif coef >  0.33 and coef <= 0.66:
            coef_class = 1
        elif coef > 0.66 and coef <= 1.0:
            coef_class = 2
        else:
            coef_class = 0
        return {"image": image, "coef_class": coef_class, "Position": Position, "selection_coeff" : coef}   

In [ ]:
class GeneticBottleneckData:
    def __init__(self,root_folder,data_table,transform= None):
        self.root_folder = root_folder
        self.data_table = data_table
        self.file_list = self._build_file_list()
        self.transform = transform
    
    def _build_file_list(self):
        file_list = []
        for image_file in os.listdir(self.root_folder):
            image_path = os.path.join(self.root_folder,image_file)
            file_list.append(image_path)
        return file_list
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self,index):
        file_path = self.file_list[index]
        image = Image.open(file_path)
        image = image.convert("RGB")
        find = re.match(r".+?[_].+?[_](.+?)[.][p][n][g]", file_path)
        coef = float(find[1])
        if self.transform:
            image = self.transform(image)
        Position = self.data_table[self.data_table["selection_coefficient"] == coef]["Position"]
        try :
            Position = Position.values[0]
        except IndexError:
            Position = 8000
        if coef <= 0.33:
             coef_class = 0
        elif coef >  0.33 and coef <= 0.66:
            coef_class = 1
        elif coef > 0.66 and coef <= 1.0:
            coef_class = 2
        else:
            coef_class = 0
        return {"image": image, "coef_class": coef_class, "Position": Position, "selection_coeff" : coef}   

In [11]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
data_selection = GeneticSelectionData("/home/jaskaran/data_images/selection", transform=transform,data_table=combined_df)



In [12]:
generator = torch.Generator().manual_seed(46)
length = len(data_selection)
training_data_len = int(0.8*length)
testing_data_len = length - training_data_len
training_data , testing_data  = random_split(data_selection, (training_data_len,testing_data_len),generator=generator)

In [13]:
training_dataloader = DataLoader(training_data, batch_size=32,shuffle=True, drop_last=False)
testing_dataloader = DataLoader(testing_data, batch_size=32,shuffle=True, drop_last=False)


In [14]:
training_data[4000]

{'image': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]),
 'coef_class': 0,
 'Position': 4622,
 'selection_coeff': 0.2383}

In [15]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, 
                               out_channels=32,
                               kernel_size=(250,10), 
                               stride= 10,
                               padding = 250)
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.MaxPool2d(kernel_size =2,
                                    stride=2)
        self.conv2 = nn.Conv2d(in_channels = 32,
                               out_channels = 64,
                               kernel_size = 2,
                               stride = 1,
                               padding = 2)
        self.fc1 = nn.Linear(432768,128)
        self.fc2 = nn.Linear(128,3)
        self.flatten = nn.Flatten(start_dim=1)
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(p=0.2)
    
    def forward(self, x):
      x = self.conv1(x)
      x = self.relu(x)
      x = self.avgpool(x)
      #x = self.conv2(x)
      #x = self.relu(x)
      x = self.flatten(x)
      x = self.fc1(x)
      x = self.relu(x)
      x = self.fc2(x)
      x = self.softmax(x)
      return x

In [16]:

model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)
model.to("cuda:1")
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    
    for batch in training_dataloader:
        images = batch['image'].to("cuda:1")
        labels = batch['coef_class'].to("cuda:1") 
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in testing_dataloader:
            images = batch['image'].to("cuda:1")
            labels = batch['coef_class'].to("cuda:1")
            output = model(images)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = correct / total 
    print(f'Epoch {epoch + 1}/{num_epochs}, loss: {loss:.2f}, Accuracy: {accuracy:.2%}')

print("Model training is completed.")

In [47]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [8]:
meta_files = os.listdir("/home/jaskaran/data_bottleneck/meta_file")
dfs = []

for file in meta_files:
    df = pd.read_csv(os.path.join("/home/jaskaran/data_bottleneck/meta_file",file),sep="\t")
    dfs.append(df)

# Concatenate all DataFrames in the dfs list into one DataFrame
combined_df_bottleneck = pd.concat(dfs, ignore_index=True)


print(combined_df_bottleneck)


      bottleneck_intensity                                         fasta_file  \
0                   0.0530  output/output_bottleneck/fasta/bottleneck_0.05...   
1                   0.0637  output/output_bottleneck/fasta/bottleneck_0.06...   
2                   0.0907  output/output_bottleneck/fasta/bottleneck_0.09...   
3                   0.0938  output/output_bottleneck/fasta/bottleneck_0.09...   
4                   0.1262  output/output_bottleneck/fasta/bottleneck_0.12...   
...                    ...                                                ...   
4945                0.9248  output/output_bottleneck/fasta/bottleneck_0.92...   
4946                0.9307  output/output_bottleneck/fasta/bottleneck_0.93...   
4947                0.9338  output/output_bottleneck/fasta/bottleneck_0.93...   
4948                0.9808  output/output_bottleneck/fasta/bottleneck_0.98...   
4949                0.9994  output/output_bottleneck/fasta/bottleneck_0.99...   

      Tajimas_D  Tajima_pi 

In [9]:
for file in os.listdir("/home/jaskaran/data_bottleneck/txt_file"):
    file_path = os.path.join("/home/jaskaran/data_bottleneck/txt_file",file)
    print(file_path)
    with open(file_path, "r") as file:
        content = file.read()
    lines = content.split("\n")

    start_index = lines.index(" The allele frequency numbers are :-  ") + 1


    end_index = next(i for i, line in enumerate(lines) if line.startswith("1. The Tajima'D for the given sequence is"))

    sfs = [num for num in lines[start_index:end_index]]
    sfs = " ".join(sfs)
    sfs = sfs[1:len(sfs)-1]
    sfs = sfs.split()
    sfs = list(map(float, sfs))
    category = [0]*9
    category[0] = sum(sfs[0:1])
    category[1] = sum(sfs[1:2])
    category[2] = sum(sfs[2:4])
    category[3] = sum(sfs[4:7])
    category[4] = sum(sfs[7:10])
    category[5] = sum(sfs[10:20])
    category[6] = sum(sfs[20:50])
    category[7] = sum(sfs[50:70])
    category[8] = sum(sfs[70:])
    find = re.match(r".+?[_].+?[_].+?[_](.+?)[.][t][x][t]",file_path)
    combined_df_bottleneck["bottleneck_intensity"] = combined_df_bottleneck["bottleneck_intensity"].astype(float)
    coef = float(find[1])
    j=1
    for i in category:
        combined_df_bottleneck.loc[combined_df_bottleneck["bottleneck_intensity"]==coef,f"category_sfs_{j}"] = i
        j+=1
    

/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.7646.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.5015.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.5572.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.4044.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.121.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.904.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.3557.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.1769.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.7853.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.2822.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.2632.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.201.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.0963.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.7011.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.6568.txt
/home/jaskaran/data_bottleneck/txt_file/bottleneck_0.7094.txt
/home/jaska

In [52]:
combined_df_bottleneck.to_csv("combined_meta_file_bottleneck.csv", index=False)